In [ ]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

# @noautodeps
# pyre-ignore-all-errors
import logging
import os
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim

from monarch.actor import Actor, current_rank, endpoint
from monarch.utils import setup_env_for_distributed
from torch.nn.parallel import DistributedDataParallel as DDP
from slurm.utils import create_slurm_job, cleanup_job


logging.basicConfig(
    level=logging.INFO,
    format="%(name)s %(asctime)s %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    force=True,
)


logger: logging.Logger = logging.getLogger(__name__)


class ToyModel(nn.Module):
    """A simple toy model for demonstration purposes."""

    def __init__(self):
        super(ToyModel, self).__init__()
        self.net1 = nn.Linear(10, 10)
        self.relu = nn.ReLU()
        self.net2 = nn.Linear(10, 5)

    def forward(self, x):
        return self.net2(self.relu(self.net1(x)))


class DDPActor(Actor):
    """This Actor wraps the basic functionality from Torch's DDP example.

    Conveniently, all of the methods we need are already laid out for us,
    so we can just wrap them in the usual Actor endpoint semantic with some
    light modifications.

    Adapted from: https://docs.pytorch.org/tutorials/intermediate/ddp_tutorial.html#basic-use-case
    """

    def __init__(self):
        self.rank = current_rank().rank

    def _rprint(self, msg):
        """Helper method to print with rank information."""
        print(f"{self.rank=} {msg}")

    @endpoint
    async def setup(self):
        """Initialize the PyTorch distributed process group."""
        self._rprint("Initializing torch distributed")

        WORLD_SIZE = int(os.environ["WORLD_SIZE"])
        # initialize the process group
        dist.init_process_group("gloo", rank=self.rank, world_size=WORLD_SIZE)
        self._rprint("Finished initializing torch distributed")

    @endpoint
    async def cleanup(self):
        """Clean up the PyTorch distributed process group."""
        self._rprint("Cleaning up torch distributed")
        dist.destroy_process_group()

    @endpoint
    async def demo_basic(self):
        """Run a basic DDP training example."""
        self._rprint("Running basic DDP example")

        # create model and move it to GPU with id rank
        local_rank = int(os.environ["LOCAL_RANK"])
        self._rprint(f"{local_rank=}")
        model = ToyModel().to(local_rank)
        ddp_model = DDP(model, device_ids=[local_rank])

        loss_fn = nn.MSELoss()
        optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)

        optimizer.zero_grad()
        outputs = ddp_model(torch.randn(20, 10))
        labels = torch.randn(20, 5).to(local_rank)
        loss_fn(outputs, labels).backward()
        optimizer.step()

        print(f"{self.rank=} Finished running basic DDP example")


async def main():
    num_nodes = 2
    gpus_per_node = 8
    mesh_name = "mesh0"
    
    # Create SLURM job
    slurm_job = create_slurm_job(mesh_name, num_nodes, gpus_per_node)

    try:
        # Get job state and create process mesh
        job_state = slurm_job.state()
        proc_mesh = job_state.mesh0.spawn_procs({"gpus": gpus_per_node})

        # Spawn DDP actor
        ddp_actor = proc_mesh.spawn("ddp_actor", DDPActor)

        # Setup distributed environment
        await setup_env_for_distributed(proc_mesh)

        # Run DDP example
        await ddp_actor.setup.call()
        await ddp_actor.demo_basic.call()
        await ddp_actor.cleanup.call()

        print("DDP example completed successfully!")

    finally:
        await cleanup_job(slurm_job)


if __name__ == "__main__":
    await main()

Found cached job at path: .monarch/job_state.pkl
SLURM job 7750 not found in queue
Cached job cannot run this spec, removing cache
Cancelled SLURM job 7750
Applying current job
Submitting SLURM job with 2 nodes
SLURM job 7751 submitted. Logs will be written to: /home/mreso/monarch/examples/slurm_7751_monarch_example_1781197.out
Saving job to cache at .monarch/job_state.pkl
Job has started, connecting to current state
SLURM job 7751 is running on 2 nodes: ['slurm-compute-node-074', 'slurm-compute-node-077']


>>> Aggregated Logs (2025-11-15 00:44:19) >>>
[8 similar log lines] [5] self.rank=5 Initializing torch distributed
<<< Aggregated Logs (2025-11-15 00:44:22) <<<

>>> Aggregated Logs (2025-11-15 00:44:22) >>>
[8 similar log lines] [0] [Gloo] Rank 0 is connected to 7 peer ranks. Expected number of connected peer ranks is : 7
[8 similar log lines] [0] self.rank=0 Finished initializing torch distributed
[8 similar log lines] [0] self.rank=0 Running basic DDP example
[8 similar log lines] [0] self.rank=0 local_rank=0
<<< Aggregated Logs (2025-11-15 00:44:25) <<<



Cancelled SLURM job 7751
slurm.utils 2025-11-15 00:44:28 INFO Job terminated successfully


DDP example completed successfully!


>>> Aggregated Logs (2025-11-15 00:44:25) >>>
[8 similar log lines] [4] self.rank=4 Finished running basic DDP example
[8 similar log lines] [0] self.rank=0 Cleaning up torch distributed
<<< Aggregated Logs (2025-11-15 00:44:28) <<<

